In [1]:
import time
import pandas as pd
import numpy as np
from numpy import array
import torch
import torch.nn as nn
from sklearn.model_selection import KFold

In [2]:
save_path = "/Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/"
model_name = "fourier_two_features"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 60

In [3]:
#### define functions ####

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, out_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, out_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [49]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 2])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 2])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 1])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 1])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 3])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 3])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 0])}")

In [4]:
# Construct sine and cosine matrices for Fourier features:
cos_matrix, sin_matrix = [[] for x in range(49)], [[] for x in range(49)]
for n in range(49):
    for k in range(49):
        cos_matrix[n].append(np.cos(2*np.pi*(n+1)*k/49))
        sin_matrix[n].append(np.sin(2*np.pi*(n+1)*k/49))

In [11]:
def dnaTwoFourier(sequence):
    seq_array = array(list(sequence))
    code = {"AA": [1], "AT": [1], "TA": [1], "TT": [1],
            "CA": [0], "CC": [0], "CG": [0], "CT": [0],
            "GA": [0], "GC": [0], "GG": [0], "GT": [0],
            "AC": [0], "TC": [0], "TG": [0], "AG": [0],
            "aa": [1], "at": [1], "ta": [1], "tt": [1],
            "ca": [0], "cc": [0], "cg": [0], "ct": [0],
            "ga": [0], "gc": [0], "gg": [0], "gt": [0],
            "ac": [0], "tc": [0], "tg": [0], "ag": [0],}
    AT_encoded_seq = []
    for i in range(len(seq_array)-1):
        AT_encoded_seq.append(code["".join((seq_array[i], seq_array[i+1]))][0])
    two_fourier_cos = np.matmul(array(AT_encoded_seq), array(cos_matrix).transpose())
    two_fourier_sin = np.matmul(array(AT_encoded_seq), array(sin_matrix).transpose())
    return list(np.concatenate((two_fourier_cos, two_fourier_sin)))

In [12]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], inv_mat))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [13]:
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append(dnaTwoFourier(sequence_nt))
X1 = np.float32(array(X1))
# X1 = X1.reshape((X1.shape[0],50,4,1))
### COMPUTE EIGENVECTORS:

# X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = np.float32(find_c0new(data_cerevisiae_nucle).astype(float))


In [14]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(dnaTwoFourier(sequence_nt))
X3 = np.float32(array(X3))
# X3 = X3.reshape((X3.shape[0],50,2,1))
# X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = np.float32(find_c0new(data_random_library).astype(float))

In [15]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaTwoFourier(sequence_nt))
X5 = np.float32(array(X5))
# X5 = X5.reshape((X5.shape[0],50,4,1))
# X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = np.float32(find_c0new(data_tiling).astype(float))

In [16]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle6.txt",delimiter = ",")
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(dnaTwoFourier(sequence_nt))
X6 = np.float32(array(X6))
# X6 = X6.reshape((X6.shape[0],50,4,1))
# X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = np.float32(find_c0new(data_chr5).astype(float))

In [17]:
m1 = np.mean(Y1)
std1 = np.std(Y1)
Z1 = (Y1-m1)/std1

m3 = np.mean(Y3)
std3 = np.std(Y3)
Z3 = (Y3-m3)/std3


m5 = np.mean(Y5)
std5 = np.std(Y5)
Z5 = (Y5-m5)/std5


m6 = np.mean(Y6)
std6 = np.std(Y6)
Z6 = (Y6-m6)/std6

In [59]:
#### random

VALIDATION_LOSS = []
fold_var = 1
n = Z3.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Z3):
    training_X = np.float32(X3[train_index])
    validation_X = np.float32(X3[val_index])
    training_Y = np.float32(Z3[train_index])
    validation_Y = np.float32(Z3[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=98, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 98)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X5)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_random.txt", index = False)

with open(save_path +model_name+"_time_random.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 1.0630
Epoch [10/60], Loss: 0.7679
Epoch [15/60], Loss: 0.7203
Epoch [20/60], Loss: 0.6929
Epoch [25/60], Loss: 0.6742
Epoch [30/60], Loss: 0.6599
Epoch [35/60], Loss: 0.6479
Epoch [40/60], Loss: 0.6382
Epoch [45/60], Loss: 0.6299
Epoch [50/60], Loss: 0.6229
Epoch [55/60], Loss: 0.6166
Epoch [60/60], Loss: 0.6111
Epoch [5/60], Loss: 1.0408
Epoch [10/60], Loss: 0.7910
Epoch [15/60], Loss: 0.7426
Epoch [20/60], Loss: 0.7135
Epoch [25/60], Loss: 0.6936
Epoch [30/60], Loss: 0.6783
Epoch [35/60], Loss: 0.6663
Epoch [40/60], Loss: 0.6565
Epoch [45/60], Loss: 0.6482
Epoch [50/60], Loss: 0.6404
Epoch [55/60], Loss: 0.6333
Epoch [60/60], Loss: 0.6266
Epoch [5/60], Loss: 0.8798
Epoch [10/60], Loss: 0.7375
Epoch [15/60], Loss: 0.7097
Epoch [20/60], Loss: 0.6923
Epoch [25/60], Loss: 0.6798
Epoch [30/60], Loss: 0.6699
Epoch [35/60], Loss: 0.6614
Epoch [40/60], Loss: 0.6536
Epoch [45/60], Loss: 0.6462
Epoch [50/60], Loss: 0.6399
Epoch [55/60], Loss: 0.6342
Epoch [60/60], Loss: 0.

In [60]:
display_fits(fits)

Average correlation on tiling: 0.5930295159282337 
Average MSE on tiling: 0.6793038725852967 
Average correlation on random: 0.5493016231856259 
Average MSE on random: 0.7014491081237793 
Average correlation on ChrV: 0.47164845281173695 
Average MSE on ChrV: 0.8494952499866486 
Average correlation on CN: 0.5935115206887798 
Average MSE on CN: 0.8720184922218323


In [61]:
#### chrv

VALIDATION_LOSS = []
fold_var = 1
n = Z6.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Z6):
    training_X = np.float32(X6[train_index])
    validation_X = np.float32(X6[val_index])
    training_Y = np.float32(Z6[train_index])
    validation_Y = np.float32(Z6[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=98, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 98)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X5)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_chrv.txt", index = False)

with open(save_path +model_name+"_time_chrv.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 1.2091
Epoch [10/60], Loss: 0.8911
Epoch [15/60], Loss: 0.8513
Epoch [20/60], Loss: 0.8330
Epoch [25/60], Loss: 0.8210
Epoch [30/60], Loss: 0.8118
Epoch [35/60], Loss: 0.8042
Epoch [40/60], Loss: 0.7979
Epoch [45/60], Loss: 0.7925
Epoch [50/60], Loss: 0.7877
Epoch [55/60], Loss: 0.7834
Epoch [60/60], Loss: 0.7796
Epoch [5/60], Loss: 1.5198
Epoch [10/60], Loss: 0.8777
Epoch [15/60], Loss: 0.8378
Epoch [20/60], Loss: 0.8179
Epoch [25/60], Loss: 0.8056
Epoch [30/60], Loss: 0.7974
Epoch [35/60], Loss: 0.7912
Epoch [40/60], Loss: 0.7862
Epoch [45/60], Loss: 0.7819
Epoch [50/60], Loss: 0.7782
Epoch [55/60], Loss: 0.7747
Epoch [60/60], Loss: 0.7714
Epoch [5/60], Loss: 1.2597
Epoch [10/60], Loss: 0.8784
Epoch [15/60], Loss: 0.8403
Epoch [20/60], Loss: 0.8196
Epoch [25/60], Loss: 0.8049
Epoch [30/60], Loss: 0.7936
Epoch [35/60], Loss: 0.7848
Epoch [40/60], Loss: 0.7776
Epoch [45/60], Loss: 0.7717
Epoch [50/60], Loss: 0.7667
Epoch [55/60], Loss: 0.7626
Epoch [60/60], Loss: 0.

In [62]:
display_fits(fits)

Average correlation on tiling: 0.5938283358539344 
Average MSE on tiling: 0.6557923197746277 
Average correlation on random: 0.5371012369290451 
Average MSE on random: 0.7216444253921509 
Average correlation on ChrV: 0.4748011714363708 
Average MSE on ChrV: 0.7759105622768402 
Average correlation on CN: 0.5943925875827899 
Average MSE on CN: 0.7102785110473633


In [67]:
#### CN

VALIDATION_LOSS = []
fold_var = 1
n = Z1.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Z1):
    training_X = np.float32(X1[train_index])
    validation_X = np.float32(X1[val_index])
    training_Y = np.float32(Z1[train_index])
    validation_Y = np.float32(Z1[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=98, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 98)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X5)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_CN.txt", index = False)

with open(save_path +model_name+"_time_CN.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 1.9453
Epoch [10/60], Loss: 0.8976
Epoch [15/60], Loss: 0.7984
Epoch [20/60], Loss: 0.7570
Epoch [25/60], Loss: 0.7316
Epoch [30/60], Loss: 0.7136
Epoch [35/60], Loss: 0.6999
Epoch [40/60], Loss: 0.6889
Epoch [45/60], Loss: 0.6795
Epoch [50/60], Loss: 0.6713
Epoch [55/60], Loss: 0.6641
Epoch [60/60], Loss: 0.6574
Epoch [5/60], Loss: 1.4342
Epoch [10/60], Loss: 0.9067
Epoch [15/60], Loss: 0.8152
Epoch [20/60], Loss: 0.7706
Epoch [25/60], Loss: 0.7424
Epoch [30/60], Loss: 0.7221
Epoch [35/60], Loss: 0.7066
Epoch [40/60], Loss: 0.6941
Epoch [45/60], Loss: 0.6836
Epoch [50/60], Loss: 0.6748
Epoch [55/60], Loss: 0.6670
Epoch [60/60], Loss: 0.6602
Epoch [5/60], Loss: 1.4603
Epoch [10/60], Loss: 0.9055
Epoch [15/60], Loss: 0.7994
Epoch [20/60], Loss: 0.7443
Epoch [25/60], Loss: 0.7101
Epoch [30/60], Loss: 0.6868
Epoch [35/60], Loss: 0.6693
Epoch [40/60], Loss: 0.6544
Epoch [45/60], Loss: 0.6416
Epoch [50/60], Loss: 0.6298
Epoch [55/60], Loss: 0.6190
Epoch [60/60], Loss: 0.

In [68]:
display_fits(fits)

Average correlation on tiling: 0.5500206456345935 
Average MSE on tiling: 0.8169593989849091 
Average correlation on random: 0.49109461931923165 
Average MSE on random: 0.8820307493209839 
Average correlation on ChrV: 0.4329616145496148 
Average MSE on ChrV: 0.978108137845993 
Average correlation on CN: 0.5597121300299535 
Average MSE on CN: 0.6956876754760742


In [69]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Y5):
    training_X = np.float32(X5[train_index])
    validation_X = np.float32(X5[val_index])
    training_Y = np.float32(Y5[train_index])
    validation_Y = np.float32(Y5[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=98, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 98)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 0.3343
Epoch [10/60], Loss: 0.1663
Epoch [15/60], Loss: 0.1455
Epoch [20/60], Loss: 0.1328
Epoch [25/60], Loss: 0.1244
Epoch [30/60], Loss: 0.1184
Epoch [35/60], Loss: 0.1141
Epoch [40/60], Loss: 0.1110
Epoch [45/60], Loss: 0.1086
Epoch [50/60], Loss: 0.1068
Epoch [55/60], Loss: 0.1054
Epoch [60/60], Loss: 0.1043
Epoch [5/60], Loss: 1.3945
Epoch [10/60], Loss: 0.2584
Epoch [15/60], Loss: 0.2095
Epoch [20/60], Loss: 0.1832
Epoch [25/60], Loss: 0.1653
Epoch [30/60], Loss: 0.1522
Epoch [35/60], Loss: 0.1424
Epoch [40/60], Loss: 0.1349
Epoch [45/60], Loss: 0.1290
Epoch [50/60], Loss: 0.1243
Epoch [55/60], Loss: 0.1205
Epoch [60/60], Loss: 0.1174
Epoch [5/60], Loss: 0.2692
Epoch [10/60], Loss: 0.1724
Epoch [15/60], Loss: 0.1486
Epoch [20/60], Loss: 0.1344
Epoch [25/60], Loss: 0.1253
Epoch [30/60], Loss: 0.1192
Epoch [35/60], Loss: 0.1149
Epoch [40/60], Loss: 0.1118
Epoch [45/60], Loss: 0.1095
Epoch [50/60], Loss: 0.1078
Epoch [55/60], Loss: 0.1064
Epoch [60/60], Loss: 0.

In [70]:
display_fits(fits)

Average correlation on tiling: 0.5465018162996097 
Average MSE on tiling: 0.11556075289845466 
Average correlation on random: 0.48127783575928235 
Average MSE on random: 0.10168596431612968 
Average correlation on ChrV: 0.4301213486764387 
Average MSE on ChrV: 0.21321986019611358 
Average correlation on CN: 0.5484018044118694 
Average MSE on CN: 0.15506017059087754
